# Rule-based scorer

Load data

In [ ]:
import pandas as pd

In [ ]:
# use the real answers as data and the predicted as test
data = pd.read_csv('Dataset.csv')
test = pd.read_csv('RULEBASED.csv')

In [ ]:
words = list(set(data['Word'].values))
test_words = list(set(test["Word"].values))

In [ ]:
test_n_words = len(test_words)
n_words = len(words); n_words

Sentence getter

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(data)
test_getter = SentenceGetter(test)

In [ ]:
sent = getter.get_next()
test_sent = test_getter.get_next()

In [ ]:
sentences = getter.sentences
test_sentences = test_getter.sentences

Define data to compare

In [ ]:
X = [[token for token, postag, label in sent] for sent in sentences]
y = [sent2labels(s) for s in sentences]
test_X = [[token for token, postag, label in sent] for sent in test_sentences]
test_y = [sent2labels(s) for s in test_sentences]

In [ ]:
y_te = y
y1_te = test_y

Scores

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
print(classification_report(y_te, y1_te))

In [ ]:
from nervaluate import Evaluator

In [ ]:
test_sents_labels = []
for sentence in test_sentences:
    sentence = [token[2] for token in sentence]
    test_sents_labels.append(sentence)

In [ ]:
sents_labels = []
for sentence in sentences:
    sentence = [token[2] for token in sentence]
    sents_labels.append(sentence)

In [ ]:
evaluator = Evaluator(sents_labels, test_sents_labels, tags= [''], loader='list')

In [ ]:
results, results_per_tag = evaluator.evaluate()

In [ ]:
results